# Veri temini

In [ ]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [ ]:
from mypyext import ml
from mypyext import extensions
from mypyext import dataanalysis as da

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
data = load_breast_cancer()

X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name="target")  # 0=malignant, 1=benign (sklearn'de böyle)

df = X.copy()
df["target"] = y

df.shape
df["target"].value_counts()


# EDA

In [ ]:
df.head()

In [ ]:
df.describe().T.sample(5)

In [ ]:
df["target"].value_counts(normalize=True).round(2)


In [ ]:
#veri tipleri beklendiği gibi mi: para birimi v.s var mı diye de bakalım
df.super_info_()

Decision Tree modelleri birçok konuda toleranslı olduklarıı için boxplot, histogram v.s çizidirmeye şuan için gerek duymuyoruz

# Modelleme

## Train/Test Ayrımı

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape


# Preprocessing işlemleri

Normalde bu aşamalr pipeline içine yedirilir. Biz açık açık görmek için şimdilik geçici olarak yapacağız. pipeline çinde X_train v.s yeniden oluşturup ilerleyeceğiz.

## Data cleaning

Yok

In [ ]:
#duplicate check for rows
len(df)-len(df.duplicated(keep=False))

In [ ]:
#duplicate check for columns
len(set(df.columns))-len(df.columns)

## imputation

superinfoda gördük, yok

## Outlier handling

outlier toleransı var, gerek dumuyoruz

## Discretization

gerek yok

## feature creation/extraction

gerek yok

## feature selection

otomatik oluyor. çok kolonumuz da olmadığı için baştan küçültmeye gerek yok

## Feature Encoding (OHE v.s)

DT'lerde gerekli değil.

## Feature Transformation

gerek yok

## Scaling

farklı skalalara toleranslı, gerek yok

## Oversampling/Undersampling (if imbalanced)

aşırı bir dengesizlik olmadığı için gerek yok

## Dimension reduction

aşırı miktarda boyut olmadığı için gerekli dğeil

# Modelleme

## İlk model (overfitting)

### Eğitim ve tahminleme

In [ ]:
tree_raw = DecisionTreeClassifier(random_state=42)
tree_raw.fit(X_train, y_train)

pred_tr = tree_raw.predict(X_train)
pred_te = tree_raw.predict(X_test)

print("Train Acc:", accuracy_score(y_train, pred_tr))
print("Test  Acc:", accuracy_score(y_test, pred_te))


In [ ]:
def clf_metrics(y_true, y_pred, y_proba=None):
    out = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
    }
    if y_proba is not None:
        out["roc_auc"] = roc_auc_score(y_true, y_proba)
    return pd.Series(out)

proba_te = tree_raw.predict_proba(X_test)[:, 1]
display(clf_metrics(y_test, pred_te, proba_te))


In [ ]:
cm = confusion_matrix(y_test, pred_te)
cm


In [ ]:
ConfusionMatrixDisplay.from_estimator(tree_raw, X_test, y_test, colorbar=False)
plt.grid(False)

In [ ]:
print(classification_report(y_test, pred_te, target_names=data.target_names))


### Görselleştirme

In [ ]:
tree_shallow = DecisionTreeClassifier(max_depth=3, random_state=42)
tree_shallow.fit(X_train, y_train)

plt.figure(figsize=(16, 8))
plot_tree(
    tree_shallow,
    feature_names=data.feature_names,
    class_names=data.target_names,
    filled=True,
    rounded=True,
    impurity=True
)
plt.show();


Kutuların içini okuyalım:

- **gini**: ilk node'da 0.468, orta saflıkta. ilk nodeoduğu için gini'nin yüksek çıkması normal. Sonrakilerde genelde düşüyor
- **samples**:  ilgili nodedaki sample sayısı. ilk nodeda 426, tüm eğitim seti. ikinci True nodunda 284, kalan 142 sağ node'da.
- **value**: sınıf sayılarıdır. ilk node'da 159 kötü huylu(malignant), 267 iyi huylu(benign)
- **class**: modelin bu node'daki kararı. ilk node'da iyi huylu kararı verilmiş, zira 267 ile iyiler çoğunlukta.

şimdi tek bir sample için karar yolu nasıl işlemiş onu görelim

In [ ]:
idx = 0
x_one = X_test.iloc[[idx]]

node_indicator = tree_shallow.decision_path(x_one)
leaf_id = tree_shallow.apply(x_one)[0]

feature = tree_shallow.tree_.feature
threshold = tree_shallow.tree_.threshold

path_nodes = node_indicator.indices[node_indicator.indptr[0]: node_indicator.indptr[1]]

rules = []
for node_id in path_nodes:
    if node_id == leaf_id:
        continue
    f = feature[node_id]
    thr = threshold[node_id]
    fname = X.columns[f]
    val = x_one.iloc[0, f]
    if val <= thr:
        rules.append(f"{fname} <= {thr:.3f}  (değer={val:.3f})")
    else:
        rules.append(f"{fname} >  {thr:.3f}  (değer={val:.3f})")

print("\nKarar Yolu:")
for i, r in enumerate(rules, 1):
    print(f"{i}. {r}")


In [ ]:
from mypyext import ml
features = X.columns
ml.tree_to_code(tree_shallow, features)

#### split görselleştirme

Verisetimiz çok boyutlu olduğu için ağacın sadece ilk split’lerinde kullanılan 2 feature’ı seçip,o split eşiklerini 2D grafikte çizdireceğiz.

In [ ]:
f1 = "worst radius"
f2 = "worst concave points"
df_plot = df[[f1, f2, "target"]]


In [ ]:
import seaborn as sns
sns.set(style="darkgrid")

plt.figure(figsize=(8,6))
sns.scatterplot(
    data=df_plot,
    x=f1,
    y=f2,
    hue="target",
    alpha=0.8
)

# tree'den manuel okuduğumuzu varsayalım
thr_f1 = 16.8   # worst radius
thr_f2 = 0.136   # worst concave points

plt.axvline(x=thr_f1, color="blue", lw=3)
plt.axhline(y=thr_f2, color="red", lw=3)

# 2.seviye

# # örnek 2. seviye split
# thr_f1_2 = 20.0
# thr_f2_2 = 0.25


# plt.axvline(x=thr_f1_2, color="blue", lw=1.5)
# plt.axhline(y=thr_f2_2, color="red", lw=1.5)


plt.title("Decision Tree Split'lerinin 2D Gösterimi")
plt.show();


### Feature importance

In [ ]:
tree_shallow.feature_importances_

In [ ]:
imp = pd.Series(tree_shallow.feature_importances_, index=X.columns).sort_values(ascending=False)
imp.head(10)

In [ ]:
imp.head(6).sort_values().plot(kind="barh")
plt.title("Top-10 Feature Importances (Shallow Tree)")
plt.show()


## overfittinge çözüm

### Mini deney

Öncelikle sadece max_depth ile oynadığımız küçük bir deney yapalım.

In [ ]:
results = []

for depth in [None, 10, 7, 5, 4, 3, 2, 1]:
    model = DecisionTreeClassifier(max_depth=depth, random_state=42)
    _ = model.fit(X_train, y_train)
    
    tr = accuracy_score(y_train, model.predict(X_train))
    te = accuracy_score(y_test, model.predict(X_test))
    fark = tr - te
    results.append((depth, tr, te, fark))

pd.DataFrame(results, columns=["max_depth", "train_acc", "test_acc","diff"])


### Cross-Validation ile daha düzgün değerlendirme

Tek split’e mahkum kalmayalım

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

model = DecisionTreeClassifier(max_depth=3, random_state=42)
scores = cross_val_score(model, X, y, cv=cv, scoring="f1")
scores.mean(), scores.std()


### GridSearch

In [ ]:
param_grid = {
    "max_depth": [2, 3, 4, 5, 6, None],
    "min_samples_leaf": [1, 2, 5, 10],
    "min_samples_split": [2, 5, 10, 20],
    "criterion": ["gini", "entropy"]
}

gs = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid=param_grid,
    cv=cv,
    scoring="f1",
    n_jobs=-1
)

gs.fit(X_train, y_train)
gs.best_params_, gs.best_score_


In [ ]:
best_tree = gs.best_estimator_

pred_te = best_tree.predict(X_test)
proba_te = best_tree.predict_proba(X_test)[:, 1]

clf_metrics(y_test, pred_te, proba_te)


### post-pruning

In [ ]:
tmp_tree = DecisionTreeClassifier(random_state=42)
path = tmp_tree.cost_complexity_pruning_path(X_train, y_train)

ccp_alphas = path.ccp_alphas
impurities = path.impurities

len(ccp_alphas)
ccp_alphas[:5]
ccp_alphas[-5:]


In [ ]:
trees = []
for a in ccp_alphas:
    t = DecisionTreeClassifier(random_state=42, ccp_alpha=a)
    _ = t.fit(X_train, y_train)
    trees.append(t)

# CV ile en iyi alpha
alpha_scores = []
for t in trees:
    scores = cross_val_score(t, X_train, y_train, cv=cv, scoring="f1")
    alpha_scores.append(scores.mean())

best_idx = int(np.argmax(alpha_scores))
best_alpha = ccp_alphas[best_idx]
best_alpha, alpha_scores[best_idx]


In [ ]:
pruned_tree = DecisionTreeClassifier(random_state=42, ccp_alpha=best_alpha)
pruned_tree.fit(X_train, y_train)

pred_te = pruned_tree.predict(X_test)
proba_te = pruned_tree.predict_proba(X_test)[:, 1]

clf_metrics(y_test, pred_te, proba_te)


In [ ]:
ml.plot_learning_curve(
    pruned_tree,
    "Öğrenme Eğrisi - Budanmış Karar Ağacı",
    X,
    y,
    cv=cv,
    train_sizes=np.linspace(0.1, 1.0, 10)
)